# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,koraput,18.8167,82.7167,14.88,51,0,1.17,IN,1703538548
1,1,smithers,54.7804,-127.1743,-1.33,88,100,1.93,CA,1703538548
2,2,kailua-kona,19.6406,-155.9956,28.11,63,20,4.63,US,1703538464
3,3,afaahiti,-17.7500,-149.2833,28.93,84,96,3.20,PF,1703538474
4,4,punta arenas,-53.1500,-70.9167,12.06,47,0,13.89,CL,1703538454


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM',
    hover_cols=['City', 'Country'], 
    color='Humidity', 
    cmap='viridis',
    size='Humidity', 
    scale=0.5,
    title='City Humidity'
)




# Display the map
plot.opts(width=800, height=600)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]
# Drop any rows with null values
ideal_weather_df_clean = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,freetown,8.4840,-13.2299,25.68,80,0,3.18,SL,1703538565
93,93,bangar,16.8936,120.4243,24.42,77,0,1.45,PH,1703538572
109,109,lihue,21.9789,-159.3672,25.77,65,0,3.60,US,1703538470
224,224,kapa'a,22.0752,-159.3190,26.18,65,0,3.60,US,1703538536
244,244,mount isa,-20.7333,139.5000,21.87,28,0,4.12,AU,1703538463


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
67,freetown,SL,8.4840,-13.2299,80,
93,bangar,PH,16.8936,120.4243,77,
109,lihue,US,21.9789,-159.3672,65,
224,kapa'a,US,22.0752,-159.3190,65,
244,mount isa,AU,-20.7333,139.5000,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel",
    "apiKey": geoapify_key, 
    "radius": radius,
    "limit": 1 #1 is for nearest hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    # Couldn't make it work, adding directly to base URL!
    #params["filter"] = f"circle:{lat},{lng},{radius}"
    #params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    #base_url = "https://api.geoapify.com/v2/places" -- Used updated based url based on geoapify documentation
    base_url = f"https://api.geoapify.com/v2/places?&lat={lat}&lon={lng}"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
freetown - nearest hotel: Formal Leona hotel
bangar - nearest hotel: Villa adelfa resort hotel and restaurant
lihue - nearest hotel: Kauai Palms
kapa'a - nearest hotel: Pono Kai Resort
mount isa - nearest hotel: Ibis Styles
darwin - nearest hotel: Mantra Pantanas Darwin
long beach - nearest hotel: Westin
chonchi - nearest hotel: Hotel Huildín
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
sur - nearest hotel: Sur Hotel
dwarka - nearest hotel: The Dwarika Hotel
diebougou - nearest hotel: Relais


,City,Country,Lat,Lng,Humidity,Hotel Name
67,freetown,SL,8.4840,-13.2299,80,Formal Leona hotel
93,bangar,PH,16.8936,120.4243,77,Villa adelfa resort hotel and restaurant
109,lihue,US,21.9789,-159.3672,65,Kauai Palms
224,kapa'a,US,22.0752,-159.3190,65,Pono Kai Resort
244,mount isa,AU,-20.7333,139.5000,28,Ibis Styles
274,darwin,AU,-12.4611,130.8418,89,Mantra Pantanas Darwin
303,long beach,US,33.7670,-118.1892,72,Westin
333,chonchi,CL,-42.6128,-73.8092,49,Hotel Huildín
408,wailua homesteads,US,22.0669,-159.3780,72,Hilton Garden Inn Kauai Wailua Bay
431,sur,OM,22.5667,59.5289,68,Sur Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,
    tiles='OSM',
    hover_cols=['City', 'Country', 'Hotel Name'],
    size='Humidity',  # Setting size based on humidity
    color='Humidity',
    colormap='viridis',
    height=500,
    width=800,
    title='City Hotels with Humidity'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)